In [1]:
'''
path = "../Data/Turkish/NE_train.txt"

file = open(path, "r", encoding="cp1254", errors="ignore")
contents = file.read().splitlines()
file.close()


filewrite = open("write", "w", encoding="utf-8")
for content in contents:
    #filewrite.write("{}\n".format(content))
    filewrite.write(content + "\n")
filewrite.close()

fileread = open("write", "r", encoding="utf-8")
texts = fileread.read().splitlines()
fileread.close()
'''

'\npath = "../Data/Turkish/NE_train.txt"\n\nfile = open(path, "r", encoding="cp1254", errors="ignore")\ncontents = file.read().splitlines()\nfile.close()\n\n\nfilewrite = open("write", "w", encoding="utf-8")\nfor content in contents:\n    #filewrite.write("{}\n".format(content))\n    filewrite.write(content + "\n")\nfilewrite.close()\n\nfileread = open("write", "r", encoding="utf-8")\ntexts = fileread.read().splitlines()\nfileread.close()\n'

In [3]:
import sys
import re

START_PATTERN = re.compile(r'^(.*?)<B_ENAMEX$', re.I)
END_SINGLE_PATTERN = re.compile(r'^TYPE="(.*?)">(.*?)<E_ENAMEX>(.*?)$', re.I)
TYPE_PATTERN = re.compile(r'^TYPE="(.*?)">(.*?)$', re.I)
END_MULTI_PATTERN = re.compile(r'^(.*?)<E_ENAMEX>(.*?)$', re.I)
EOS_PATTERN = re.compile(r'^([^<>]*)\.?\t(\d+)$', re.I)
NON_ENTITY_TYPE = 'O'


def check_and_process_eos(token):
    match = re.match(EOS_PATTERN, token)
    if match:
        out.write(match.group(1) + '\t' + cur_type + '\n')
        out.write('.' + '\t' + cur_type + '\n')
        out.write('\n')
        return True
    return False

infile  = "../Data/Turkish/NE_train.txt"
outfile = "BIO.txt"
cur_type = NON_ENTITY_TYPE
with open(infile, "r", encoding="cp1254", errors="ignore") as f, open(outfile, 'w', encoding="utf-8") as out:
    for line in f:
        for token in line.strip().split(' '):
            token = token.strip()
            if not token:
                continue

            match = re.match(START_PATTERN, token)
            if match:
                if match.group(1):
                    out.write(match.group(1) + '\t' + NON_ENTITY_TYPE + '\n')
                continue

            match = re.match(END_SINGLE_PATTERN, token)
            if match:
                out.write(match.group(2) + '\t' + match.group(1) + '\n')
                cur_type = NON_ENTITY_TYPE
                if not check_and_process_eos(match.group(3)):
                    out.write(match.group(3) + '\t' + cur_type + '\n')
                continue

            match = re.match(TYPE_PATTERN, token)
            if match:
                cur_type = match.group(1)
                out.write(match.group(2) + '\t' + cur_type + '\n')
                continue

            match = re.match(END_MULTI_PATTERN, token)
            if match:
                out.write(match.group(1) + '\t' + cur_type + '\n')
                cur_type = NON_ENTITY_TYPE
                if not check_and_process_eos(match.group(2)):
                    out.write(match.group(2) + '\t' + cur_type + '\n')
                continue

            if check_and_process_eos(token):
                continue

            out.write(token + '\t' + cur_type + '\n')
        out.write("" + '\n')

In [8]:
import pandas as pd

df = pd.read_csv(outfile, sep= "\t", header=None)
df.columns = ["words", "labels"]

In [10]:
df.head(20)

,words,labels
0,Müzik,O
1,Şenliği,O
2,'ne,O
3,hazırlanın,O
4,POZİTİF,ORGANIZATION
5,ve,ORGANIZATION
6,Açık,ORGANIZATION
7,Radyo,ORGANIZATION
8,NaN,O
9,işbirliğiyle,O
